# [7] 視界スコア指標 - Phase 5

ward座標データからグリッド特徴量を生成し、勝敗予測モデルを学習します。

## 処理フロー
1. **Task A**: グリッド特徴量生成（wards_matched.csv → ward_grid.npz）
2. **Task B**: データセット構築（ward_grid.npz + 勝敗ラベル → vision_dataset.npz）
3. **Task C**: 予測モデル学習（ロジスティック回帰 / CNN）
4. **Task D**: ヒートマップ可視化

## 前提条件
- **06_ward_batch_processing.ipynb 完了**（wards_matched.csvが各試合フォルダに存在）
- タイムラインデータ（data/timeline/*.json）

In [ ]:
#cell-1: ライブラリインポート
from pathlib import Path
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt

PROJECT_ROOT = Path(r"c:\Users\lapis\Desktop\LoL_WorkSp_win\pyLoL-_WorkSp\pyLoL-v2")

# autoLeagueモジュール
import sys
sys.path.insert(0, str(PROJECT_ROOT))

from autoLeague.scoring import (
    DatasetBuilder,
    build_dataset,
    VisionPredictor,
    load_dataset,
    visualize_importance_heatmap,
    visualize_importance_grid,
    visualize_top_cells,
    print_importance_summary,
)

print("インポート完了")

In [ ]:
#cell-2: パス設定

# 入力データ
DATASET_DIR = Path(r"C:\dataset_20260105")  # ミニマップキャプチャ + wards_matched.csv
TIMELINE_DIR = PROJECT_ROOT / "data" / "timeline"  # タイムラインJSON

# 出力
OUTPUT_DATASET = PROJECT_ROOT / "data" / "vision_dataset.npz"
OUTPUT_MODEL_DIR = PROJECT_ROOT / "models"
OUTPUT_HEATMAP_DIR = PROJECT_ROOT / "heatmaps"

# 確認
print(f"データセットディレクトリ: {DATASET_DIR}")
print(f"  存在: {DATASET_DIR.exists()}")

if DATASET_DIR.exists():
    match_dirs = sorted(DATASET_DIR.glob("JP1-*"))
    print(f"  試合数: {len(match_dirs)}")

print(f"\nタイムラインディレクトリ: {TIMELINE_DIR}")
print(f"  存在: {TIMELINE_DIR.exists()}")

if TIMELINE_DIR.exists():
    timeline_files = list(TIMELINE_DIR.glob("JP1_*.json"))
    print(f"  ファイル数: {len(timeline_files)}")

## Task A + B: データセット構築

各試合の`wards_matched.csv`からグリッド特徴量を生成し、勝敗ラベルを付与して統合データセットを作成します。

In [ ]:
#cell-3: wards_matched.csvの確認

# 最初の試合のwards_matched.csvを確認
if DATASET_DIR.exists():
    match_dirs = sorted(DATASET_DIR.glob("JP1-*"))
    if match_dirs:
        sample_csv = match_dirs[0] / "wards_matched.csv"
        if sample_csv.exists():
            df = pd.read_csv(sample_csv)
            print(f"サンプル: {sample_csv}")
            print(f"レコード数: {len(df)}")
            print(f"\nカラム: {list(df.columns)}")
            print(f"\n先頭5行:")
            display(df.head())
        else:
            print(f"wards_matched.csvが見つかりません: {sample_csv}")

In [ ]:
#cell-4: データセット構築（Task A + B）

# DatasetBuilderを使用
builder = DatasetBuilder(
    dataset_dir=DATASET_DIR,
    timeline_dir=TIMELINE_DIR,
)

# 全試合を処理
print("データセット構築開始...")
X, y, match_ids = builder.build()

print(f"\n=== データセット情報 ===")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"試合数: {len(match_ids)}")
print(f"Blue勝利: {y.sum()}, Red勝利: {len(y) - y.sum()}")

In [ ]:
#cell-5: データセット保存

# 出力ディレクトリ作成
OUTPUT_DATASET.parent.mkdir(parents=True, exist_ok=True)

# 保存
np.savez(
    OUTPUT_DATASET,
    X=X,
    y=y,
    match_ids=np.array(match_ids, dtype=object)
)

print(f"データセット保存完了: {OUTPUT_DATASET}")
print(f"ファイルサイズ: {OUTPUT_DATASET.stat().st_size / 1024:.1f} KB")

## Task C: 予測モデル学習

グリッド特徴量から勝敗を予測するモデルを学習します。

### モデル選択肢
1. **ロジスティック回帰**（推奨）: シンプルで解釈しやすい
2. **浅いCNN**: 空間パターンを学習可能

In [ ]:
#cell-6: データセット読み込み（既存データを使用する場合）

# 既にvision_dataset.npzがある場合はここから開始可能
if OUTPUT_DATASET.exists():
    X, y, match_ids = load_dataset(OUTPUT_DATASET)
    print(f"データセット読み込み完了")
    print(f"X shape: {X.shape}")
    print(f"y shape: {y.shape}")
    print(f"Blue勝利: {y.sum()}, Red勝利: {len(y) - y.sum()}")
else:
    print(f"データセットが見つかりません: {OUTPUT_DATASET}")
    print("cell-4, cell-5を先に実行してください")

In [ ]:
#cell-7: Train/Test分割

from sklearn.model_selection import train_test_split

TEST_SIZE = 0.2
RANDOM_STATE = 42

try:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=TEST_SIZE,
        random_state=RANDOM_STATE,
        stratify=y
    )
except ValueError:
    # サンプル数が少ない場合は層化なし
    print("警告: サンプル数が少ないため層化抽出を無効化")
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=TEST_SIZE,
        random_state=RANDOM_STATE
    )

print(f"学習データ: {len(X_train)}件")
print(f"テストデータ: {len(X_test)}件")

In [ ]:
#cell-8: ロジスティック回帰モデルの学習

# モデル作成
predictor_lr = VisionPredictor(model_type="logistic")

# 学習
print("ロジスティック回帰モデル学習中...")
metrics_lr = predictor_lr.fit(X_train, y_train, verbose=True)

print(f"\n学習完了")
print(f"Train accuracy: {metrics_lr['train_accuracy']:.3f}")

In [ ]:
#cell-9: ロジスティック回帰モデルの評価

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# 予測
y_pred_lr = predictor_lr.predict(X_test)

# 評価
print("=== ロジスティック回帰 評価結果 ===")
print(f"Accuracy:  {accuracy_score(y_test, y_pred_lr):.3f}")
print(f"Precision: {precision_score(y_test, y_pred_lr, zero_division=0):.3f}")
print(f"Recall:    {recall_score(y_test, y_pred_lr, zero_division=0):.3f}")
print(f"F1 Score:  {f1_score(y_test, y_pred_lr, zero_division=0):.3f}")

# 混同行列
cm = confusion_matrix(y_test, y_pred_lr)
print(f"\n混同行列:")
print(f"(予測→)   Red  Blue")
print(f"実際Red   {cm[0][0]:4d}  {cm[0][1]:4d}")
print(f"実際Blue  {cm[1][0]:4d}  {cm[1][1]:4d}")

In [ ]:
#cell-10: CNNモデルの学習（オプション）

# CNNモデル作成
predictor_cnn = VisionPredictor(model_type="cnn")

# 学習
print("CNNモデル学習中...")
metrics_cnn = predictor_cnn.fit(X_train, y_train, epochs=100, verbose=True)

# 予測
y_pred_cnn = predictor_cnn.predict(X_test)

# 評価
print("\n=== CNN 評価結果 ===")
print(f"Accuracy:  {accuracy_score(y_test, y_pred_cnn):.3f}")
print(f"Precision: {precision_score(y_test, y_pred_cnn, zero_division=0):.3f}")
print(f"Recall:    {recall_score(y_test, y_pred_cnn, zero_division=0):.3f}")
print(f"F1 Score:  {f1_score(y_test, y_pred_cnn, zero_division=0):.3f}")

In [ ]:
#cell-11: モデル比較

print("=== モデル比較 ===")
print(f"{'モデル':<20} {'Accuracy':<12} {'F1 Score':<12}")
print("-" * 44)
print(f"{'ロジスティック回帰':<20} {accuracy_score(y_test, y_pred_lr):<12.3f} {f1_score(y_test, y_pred_lr, zero_division=0):<12.3f}")
print(f"{'CNN':<20} {accuracy_score(y_test, y_pred_cnn):<12.3f} {f1_score(y_test, y_pred_cnn, zero_division=0):<12.3f}")
print(f"{'ベースライン(ランダム)':<20} {'0.500':<12} {'-':<12}")

# 推奨モデル選択
if accuracy_score(y_test, y_pred_lr) >= accuracy_score(y_test, y_pred_cnn):
    best_model = "logistic"
    best_predictor = predictor_lr
else:
    best_model = "cnn"
    best_predictor = predictor_cnn

print(f"\n推奨モデル: {best_model}")

In [ ]:
#cell-12: モデル保存

OUTPUT_MODEL_DIR.mkdir(parents=True, exist_ok=True)

# ロジスティック回帰モデル保存
lr_path = OUTPUT_MODEL_DIR / "vision_predictor.joblib"
predictor_lr.save(lr_path)
print(f"ロジスティック回帰モデル保存: {lr_path}")

# CNNモデル保存
cnn_path = OUTPUT_MODEL_DIR / "vision_predictor.pt"
predictor_cnn.save(cnn_path)
print(f"CNNモデル保存: {cnn_path}")

# メトリクス保存
metrics_path = OUTPUT_MODEL_DIR / "vision_predictor_metrics.json"
metrics = {
    "model_type": best_model,
    "n_samples_train": len(X_train),
    "n_samples_test": len(X_test),
    "test_accuracy": float(accuracy_score(y_test, y_pred_lr if best_model == "logistic" else y_pred_cnn)),
    "test_precision": float(precision_score(y_test, y_pred_lr if best_model == "logistic" else y_pred_cnn, zero_division=0)),
    "test_recall": float(recall_score(y_test, y_pred_lr if best_model == "logistic" else y_pred_cnn, zero_division=0)),
    "test_f1": float(f1_score(y_test, y_pred_lr if best_model == "logistic" else y_pred_cnn, zero_division=0)),
}
with open(metrics_path, "w") as f:
    json.dump(metrics, f, indent=2)
print(f"メトリクス保存: {metrics_path}")

## Task D: ヒートマップ可視化

特徴量重要度をヒートマップとして可視化し、「どの座標・時間帯のward配置が勝敗に影響するか」を分析します。

In [ ]:
#cell-13: 特徴量重要度取得

# ロジスティック回帰の重要度を使用（解釈しやすい）
importance = predictor_lr.get_feature_importance()

print(f"特徴量重要度")
print(f"  形状: {importance.shape}")
print(f"  値域: [{importance.min():.4f}, {importance.max():.4f}]")

# 統計情報表示
print_importance_summary(importance)

In [ ]:
#cell-14: 特徴量重要度保存

importance_path = OUTPUT_MODEL_DIR / "vision_importance.npy"
np.save(importance_path, importance)
print(f"特徴量重要度保存: {importance_path}")

In [ ]:
#cell-15: 時間帯別ヒートマップ生成

OUTPUT_HEATMAP_DIR.mkdir(parents=True, exist_ok=True)

# ヒートマップ生成
output_files = visualize_importance_heatmap(
    importance=importance,
    output_dir=OUTPUT_HEATMAP_DIR,
)

print(f"生成されたファイル:")
for f in output_files:
    print(f"  {f}")

In [ ]:
#cell-16: グリッド表示（3時間帯並列）

grid_path = visualize_importance_grid(
    importance=importance,
    output_path=OUTPUT_HEATMAP_DIR / "importance_grid.png",
)

# 表示
from IPython.display import Image, display
display(Image(filename=str(grid_path), width=900))

In [ ]:
#cell-17: 上位セルハイライト表示

top_cells_path = visualize_top_cells(
    importance=importance,
    output_path=OUTPUT_HEATMAP_DIR / "importance_top_cells.png",
    top_n=10,
)

# 表示
display(Image(filename=str(top_cells_path), width=600))

In [ ]:
#cell-18: 統合ヒートマップ表示

combined_path = OUTPUT_HEATMAP_DIR / "importance_combined.png"
if combined_path.exists():
    display(Image(filename=str(combined_path), width=600))

## 結果サマリー

In [ ]:
#cell-19: 結果サマリー

print("=" * 60)
print("Phase 5 視界スコア指標 - 完了")
print("=" * 60)

print(f"\n【データセット】")
print(f"  試合数: {len(match_ids)}")
print(f"  入力形状: {X.shape}")
print(f"  Blue勝利: {y.sum()}, Red勝利: {len(y) - y.sum()}")

print(f"\n【モデル性能】")
print(f"  推奨モデル: {best_model}")
print(f"  Test Accuracy: {metrics['test_accuracy']:.1%}")
print(f"  Test F1 Score: {metrics['test_f1']:.1%}")
print(f"  ベースライン改善: +{(metrics['test_accuracy'] - 0.5) / 0.5 * 100:.1f}%")

print(f"\n【出力ファイル】")
print(f"  データセット: {OUTPUT_DATASET}")
print(f"  モデル: {lr_path}")
print(f"  メトリクス: {metrics_path}")
print(f"  ヒートマップ: {OUTPUT_HEATMAP_DIR}/")

print(f"\n【主要な知見】")
phase_totals = [np.abs(importance[i]).sum() for i in range(3)]
most_important_phase = np.argmax(phase_totals)
phase_names = ["Phase 1 (0-10min)", "Phase 2 (10-20min)", "Phase 3 (20min+)"]
print(f"  最重要時間帯: {phase_names[most_important_phase]}")